# RAG 基礎入門

本範例展示：
1. **第1個儲存格**：建立向量資料庫
2. **第2個儲存格**：查詢向量資料庫
3. **第3個儲存格**: 整合chain的功能

學習目標：理解如何將文本轉換為向量並進行相似度檢索

In [ ]:
# 第1個儲存格：建立向量資料庫

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_jina")

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量資料庫...")

    # 確保文字檔案存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"檔案 {file_path} 不存在。請檢查路徑。"
        )

    # 從檔案讀取文字內容
    loader = TextLoader(file_path)
    documents = loader.load()

    # 將文件分割成塊
    # chunk_size=1000: 每個文本區塊最多 1000 個字元
    # chunk_overlap=0: 區塊之間不重疊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")
    print(f"範例塊:\n{docs[0].page_content}\n")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    print("使用 Jina Embeddings v2（繁體中文開源模型）")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立向量存儲並自動持久化
    print("\n--- 正在建立向量存儲 ---")
    
    # 使用 PersistentClient 以避免權限問題
    client = chromadb.PersistentClient(path=persistent_directory)
    
    db = Chroma.from_documents(
        docs, 
        embeddings, 
        client=client,
        collection_name="smartphone_manual"
    )
    print("\n--- 完成建立向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

In [ ]:
# 第2個儲存格：查詢向量資料庫

import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_jina")

# 定義嵌入模型（使用開源繁體中文模型）
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用 PersistentClient 載入現有的向量存儲
client = chromadb.PersistentClient(path=persistent_directory)

db = Chroma(
    client=client,
    collection_name="smartphone_manual",
    embedding_function=embeddings
)

# 定義使用者的問題
query = "如何設定指紋辨識？"

# 根據查詢檢索相關文件
# search_type="similarity": 使用相似度搜尋（預設方法）
# k=3: 返回最相關的 3 個文件
# 注意：由於某些 embedding 模型的相似度分數可能不在 0-1 範圍內，
#      因此使用簡單的 similarity 搜尋而非 similarity_score_threshold
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

In [ ]:
# 第3個儲存格：整合 Chain - 建立簡單的 RAG 問答鏈

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義使用者的問題
question = "如何設定指紋辨識？"

print(f"問題: {question}\n")

# 步驟 1: 從向量資料庫檢索相關文件
print("=" * 60)
print("步驟 1: 從向量資料庫檢索相關文件")
print("=" * 60)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}  # 只取最相關的 2 個文件
)
retrieved_docs = retriever.invoke(question)

print(f"找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"文件 {i} (前100字):")
    print(doc.page_content[:100] + "...\n")

# 步驟 2: 建立 RAG Chain
print("=" * 60)
print("步驟 2: 建立 RAG Chain")
print("=" * 60)

# 定義提示模板
template = """你是一個智慧型手機的客服助手。請根據以下參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

請用繁體中文回答，並且：
1. 只根據參考資料回答，不要編造內容
2. 如果參考資料中沒有答案，請誠實說「我在資料中找不到相關資訊」
3. 回答要清楚、具體、有條理

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 建立 LLM（使用本地 Ollama）
llm = ChatOllama(model="llama3.2", temperature=0)

# 定義文件格式化函數
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 建立完整的 RAG Chain
# RunnablePassthrough() 讓 question 直接傳遞下去
# retriever | format_docs 將檢索結果格式化為 context
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 建立完成\n")

# 步驟 3: 執行 RAG Chain 並取得答案
print("=" * 60)
print("步驟 3: 執行 RAG Chain")
print("=" * 60)

answer = rag_chain.invoke(question)

print(f"\n【AI 回答】\n{answer}\n")

print("=" * 60)
print("RAG Chain 流程總結")
print("=" * 60)
print("1. 使用者提問 → 向量檢索找相關文件")
print("2. 將文件和問題組合成提示詞")
print("3. 送給 LLM 生成答案")
print("4. 解析並返回答案")
print("\n💡 這就是從 Chain 3 學到的技巧應用在 RAG 上！")